In [1]:
import os
import tensorflow as tf
import cv2
import numpy as np
import json
import shutil
from collections import deque
from model_architecture import build_tools
from utils.image_folder import make_dataset
from PIL import Image
from utils.utils import data_tools
from utils.FPS import FPS
from config import *
from tqdm import tqdm

from packaging import version
from tensorflow import __version__ as tfver
using_tf2 = version.parse(tfver) >= version.parse("2.0.0")

/home/pame/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pame/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pame/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pame/anaconda3/envs/tf1/lib/python3.7/site-packages/ten

In [2]:
# Allow memory growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
    if not using_tf2:
        session = tf.Session(config=config)
    print('\n'*2 + '--Memory Growth enabled--' + '\n'*2)

#Función para encontrar un path
def get_folder(path):
    folder = '/'
    for x in path.split('/')[1:-1]:
        folder = folder + x + '/'
    return folder




--Memory Growth enabled--




In [3]:
#Para que lea todas las imágenes del dataset completo:
dataset_dir = '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST'
output_dir = './files/test_set/'
name = '{}_{}_{}_e{}_ns{}.npz'.format(model_name, data_type, vers, test_epoch, nsequences)
output_file = os.path.join(output_dir, name)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
# *********** 1.A  TEST   ************
# ALCOHOL
alcohol_dir = os.path.join(dataset_dir, 'Test_alcohol')
alcohol_dataset = make_dataset(alcohol_dir)

# DRUG
drug_dir = os.path.join(dataset_dir, 'Test_drug')
drug_dataset = make_dataset(drug_dir)
# SLEEP
sleep_dir = os.path.join(dataset_dir, 'Test_sleep')
sleep_dataset = make_dataset(sleep_dir)
# CONTROL
control_dir = os.path.join(dataset_dir, 'Test_control')
control_dataset = make_dataset(control_dir)

#Listado de carpetas que hay
def carpetas (dataset):
    folders = []
    for archivo in dataset:
        folders.append(get_folder(archivo))
    folders = np.unique(folders)
    return(folders)

# *********** 2.A TEST FOLDERS ***********
folders_alcohol = carpetas((alcohol_dataset))
folders_drug = carpetas(drug_dataset)
folders_sleep = carpetas(sleep_dataset)
folders_control = carpetas(control_dataset)

# ********* Create model ***********
model_tools = build_tools()
network = model_tools.create_network(model_name)
print('\n--Network created successfully--\n')

#Load weights 
network.load_weights(checkpoint_path.format(epoch=test_epoch))
print('\n--Weights loaded successfully--\n')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 140, 210, 3)]  0         
_________________________________________________________________
time_distributed (TimeDistri (None, 8, 140, 210, 64)   1792      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 140, 210, 64)   256       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 8, 70, 105, 64)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 8, 70, 105, 128)   73856     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 8, 70, 105, 128)   512       
_________

In [5]:
# ****** FUNCIONES  *******
def inference (network, folder, clase = [0,1], tipo = 0, nsequences = None):
    predictions = []
    clase = np.array(clase, dtype='uint8')
    tipo = np.array(tipo, dtype='uint8')
    for register in tqdm(folder, desc = "Making predictions: "):
        sequence_dataset = make_dataset(str(register))
        sequence_dataset = sort_names(sequence_dataset)
        #print('SHAPE DE SEQUENCE DATASET',  np.shape(sequence_dataset))

        if nsequences == 'all':
            nsequences = len(sequence_dataset) - 8
        else:
            nsequences = min([nsequences, len(sequence_dataset) - 8])

        for i in range(nsequences):
            seq_tensor = []
            ind_1 = i
            ind_2 = ind_1+8
            for j in range(ind_1, ind_2):

                img = Image.open(sequence_dataset[j]).convert('RGB').resize((210,140))
                #Debería ser un (140,210,3)
                img = np.array(img)
                # adquiere el (8,140,210,3)
                seq_tensor.append(img)
            
            np_image_seqs = np.reshape(np.array(seq_tensor)/255,(1,time,height,width,color_channels))
            r = network.predict(np_image_seqs)
            r = np.squeeze(r, 0)

            data = [r, clase, tipo]
            predictions.append(data)
            #print('PREDICTIONS',predictions)
            print('SHAPE DE SEQUENCE DATASET',  np.shape(sequence_dataset))
            print('TAMAÑO DE PREDICTIONS', len(predictions))
            print('SHAPE DE PREDICTIONS', np.shape(predictions))
    # tamaño final (N,8,140,210,3)
    return(predictions)


In [7]:
# *************** 4.A  TEST FINAL **************
# No control
print('Alcohol:')
if data_type == 'iris_2classes':
    clase = [1,0]
    tipo = 0
if data_type == 'iris_4classes':
    clase = [1,0,0,0]
    tipo = 0


pred_alcohol = inference (network, folders_alcohol, clase = clase, tipo = tipo, nsequences = nsequences)


Alcohol:


Making predictions:   4%|▍         | 3/72 [00:00<00:03, 22.30it/s]

SHAPE DE SEQUENCE DATASET (149,)
TAMAÑO DE PREDICTIONS 1
SHAPE DE PREDICTIONS (1, 3)
SHAPE DE SEQUENCE DATASET (101,)
TAMAÑO DE PREDICTIONS 2
SHAPE DE PREDICTIONS (2, 3)
SHAPE DE SEQUENCE DATASET (150,)
TAMAÑO DE PREDICTIONS 3
SHAPE DE PREDICTIONS (3, 3)
SHAPE DE SEQUENCE DATASET (119,)
TAMAÑO DE PREDICTIONS 4
SHAPE DE PREDICTIONS (4, 3)
SHAPE DE SEQUENCE DATASET (150,)
TAMAÑO DE PREDICTIONS 5
SHAPE DE PREDICTIONS (5, 3)


Making predictions: 100%|██████████| 72/72 [00:00<00:00, 180.61it/s]

SHAPE DE SEQUENCE DATASET (118,)
TAMAÑO DE PREDICTIONS 6
SHAPE DE PREDICTIONS (6, 3)
SHAPE DE SEQUENCE DATASET (129,)
TAMAÑO DE PREDICTIONS 7
SHAPE DE PREDICTIONS (7, 3)
SHAPE DE SEQUENCE DATASET (107,)
TAMAÑO DE PREDICTIONS 8
SHAPE DE PREDICTIONS (8, 3)
SHAPE DE SEQUENCE DATASET (150,)
TAMAÑO DE PREDICTIONS 9
SHAPE DE PREDICTIONS (9, 3)


In [13]:
folder = folders_alcohol

# ****** FUNCIONES  *******

predictions = []
clase = np.array(clase, dtype='uint8')
tipo = np.array(tipo, dtype='uint8')
#for register in folder:
register = folder[0]
sequence_dataset = make_dataset(str(register))
sequence_dataset = sort_names(sequence_dataset)
#print('SHAPE DE SEQUENCE DATASET',  np.shape(sequence_dataset))

if nsequences == 'all':
    nsequences = len(sequence_dataset) - 8
else:
    nsequences = min([nsequences, len(sequence_dataset) - 8])

for i in range(nsequences):
    seq_tensor = []
    ind_1 = i
    ind_2 = ind_1+8
    for j in range(ind_1, ind_2):

        img = Image.open(sequence_dataset[j]).convert('RGB').resize((210,140))
        #Debería ser un (140,210,3)
        img = np.array(img)
        # adquiere el (8,140,210,3)
        seq_tensor.append(img)

    np_image_seqs = np.reshape(np.array(seq_tensor)/255,(1,time,height,width,color_channels))
    r = network.predict(np_image_seqs)
    r = np.squeeze(r, 0)

    data = [r, clase, tipo]
    predictions.append(data)
    #print('PREDICTIONS',predictions)
    print('SHAPE DE SEQUENCE DATASET',  np.shape(sequence_dataset))
    print('TAMAÑO DE PREDICTIONS', len(predictions))
    print('SHAPE DE PREDICTIONS', np.shape(predictions))
# tamaño final (N,8,140,210,3)



In [41]:
#Función para que imágenes salgan en orden correcto
def sort_names(image_names):
    # Find Indices
    idx1 = [int(name.split('_')[-1].split('.')[0]) for name in image_names]
    idx2 = [int(name.split('_')[-2].split('.')[0]) for name in image_names]
    
    unique_idx1 = np.unique(idx1)
    unique_idx2 = np.unique(idx2)
    
    if len(unique_idx1) > len(unique_idx2):
        idx = idx1
    else:
        idx = idx2

    # Create empty list to store the sorted names
    sorted_names = [None]*(max(idx)+1)

    # Sort names
    for i, j in enumerate(idx):
        sorted_names[j] = image_names[i]

    # Eliminate empty elements
    sorted_names = [name for name in sorted_names if name is not None]

    return sorted_names

In [42]:
register = folder[17]
sequence_dataset = make_dataset(str(register))

sequence_dataset = sort_names(sequence_dataset)
sequence_dataset

['/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_0_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_1_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_2_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_3_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_4_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_5_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_6_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_7_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_8_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/30_1336/B_57_9_2020.jpg',
 '/home/pame/PROYECT1/Vehicle_

In [37]:
path = sequence_dataset[0]
aux = path.split('_')
aux

['/home/pame/PROYECT1/Vehicle',
 'Collision-master/files/TEST/Test',
 'alcohol/E',
 '4',
 '1',
 '1',
 'R',
 '5/E',
 '4',
 '1',
 '1',
 'R',
 'M',
 'N',
 'N',
 '1996',
 '0',
 '2017.bmp']

In [38]:
path

'/home/pame/PROYECT1/Vehicle_Collision-master/files/TEST/Test_alcohol/E_4_1_1_R_5/E_4_1_1_R_M_N_N_1996_0_2017.bmp'

In [39]:
aux[-1]

'2017.bmp'

In [40]:
aux[-1].split('.')

['2017', 'bmp']